In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from datetime import datetime
import re

# Initialize the WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

# Function to extract RTC data
def extract_rtc_data():
    driver.get("https://www.redbus.in/online-booking/rtc-directory")
    driver.maximize_window()

    # Wait for the RTC elements to load
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "D113_item_rtc")))

    # Extract all elements with the class 'D113_item_rtc' that is all state buses name
    rtc_elements = driver.find_elements(By.CLASS_NAME, "D113_item_rtc")

    # Initialize a list to store the RTC details
    rtc_data = []

    # find state buses and link
    for rtc in rtc_elements:
        try:
            a_tag = rtc.find_element(By.TAG_NAME, 'a')
            link = a_tag.get_attribute('href')
            name = a_tag.text.strip()
            rtc_data.append({'Name': name, 'Link': link})
        except Exception as e:
            print("Error:", e)

    return rtc_data

#to find there is elemnts  based on 'route_link' class
def filter_rtc_data(rtc_data):
    filtered_rtc_data = []
    for item in rtc_data:
        try:
            driver.get(item['Link'])
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))  # Wait for the page to load

            # Check if the page has an element with class name 'route_link'
            route_link_elements = driver.find_elements(By.CLASS_NAME, 'route_link')
            if route_link_elements:
                filtered_rtc_data.append(item)
        except Exception as e:
            print("Error:", e)

    return filtered_rtc_data

# Function to scroll to the bottom of the page
def scroll_to_bottom(driver):
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        body = driver.find_element(By.TAG_NAME, "body")
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)  # Wait for new content to load
        new_page_source = driver.page_source
        if old_page_source == new_page_source:
            scrolling = False

# Function to extract all bus items from a given route link
def extract_all_bus_items(driver, wait, route_link, route_title, transport):
    driver.get(route_link)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'travels')))
    time.sleep(2)

    # Extract the search date 
    try:
        search_date_element = driver.find_element(By.XPATH, "//*[@id='searchDat']")
        search_date_str = search_date_element.get_attribute("value") + " 2024"
        search_date = datetime.strptime(search_date_str, "%d %b %Y").strftime("%Y-%m-%d")
    except Exception as e:
        search_date = None
        print("Error:", e)

    scroll_to_bottom(driver)

    bus_items = []

    try:
        # Check for "View Buses" button
        view_buses_buttons = driver.find_elements(By.XPATH, '//div[@class="clearfix"]//div[@class="button" and contains(text(), "View Buses")]')
        if view_buses_buttons:
            for view_buses_button in view_buses_buttons:
                try:
                    driver.execute_script("arguments[0].scrollIntoView();", view_buses_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", view_buses_button)
                    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'travels')))
                    time.sleep(2)
                except Exception as e:
                    continue

        # Extract bus elements including "View Buses" button
        bus_elements = driver.find_elements(By.CLASS_NAME, 'clearfix.bus-item')
        for bus in bus_elements:
            # Extract seat availability text
            seat_left_text = bus.find_element(By.CLASS_NAME, 'seat-left').text
            
            # Use regular expression to find the first integer in the text
            match = re.search(r'\d+', seat_left_text)
            seat_left = int(match.group(0)) if match else 0
            
            details = {
                'Transport': transport,
                'route_name': route_title,
                'route_link': route_link,
                'Date': search_date,  # extracted date with year 2024 in date format
                'busname': bus.find_element(By.CLASS_NAME, 'travels').text,
                'bustype': bus.find_element(By.CLASS_NAME, 'bus-type').text,
                'departing_time': bus.find_element(By.CLASS_NAME, 'dp-time').text,
                'duration': bus.find_element(By.CLASS_NAME, 'dur').text,
                'bp_time': bus.find_element(By.CLASS_NAME, 'bp-time').text,
                'star_rating': bus.find_element(By.CLASS_NAME, 'rating').text,
                'price': int(bus.find_element(By.CSS_SELECTOR, '.seat-fare .fare.d-block .f-19.f-bold').text),
                'seats_available': seat_left,
            }
            bus_items.append(details)
    except Exception as e:
        pass
    return bus_items

# Extract and filter RTC data
rtc_data = extract_rtc_data()
filtered_rtc_data = filter_rtc_data(rtc_data)

# Initialize a list to store all bus details
all_bus_details = []

print("Loading, please wait...")

# Extract bus details for each filtered RTC data link and store in the list
try:
    for route in filtered_rtc_data:
        route_title = route['Name']
        route_link = route['Link']

        try:
            driver.get(route_link)

            # Scroll to the bottom to load all items
            scroll_to_bottom(driver)

            # Pagination handling
            all_routes = []
            while True:
                ktcl_routes = driver.find_elements(By.CLASS_NAME, "route")
                for route in ktcl_routes:
                    title = route.text
                    link = route.get_attribute('href')
                    if title and link:
                        all_routes.append({"title": title, "link": link, "Transport": route_title})

                try:
                    pagination_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "DC_117_paginationTable")))
                    current_page = pagination_container.find_element(By.CLASS_NAME, "DC_117_pageActive").text
                    next_page_number = int(current_page) + 1
                    next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{next_page_number}"]')

                    actions = ActionChains(driver)
                    actions.move_to_element(next_page_button).perform()
                    time.sleep(1)

                    next_page_button.click()
                    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route")))
                    time.sleep(2)

                except Exception as e:
                    break

            for route in all_routes:
                route_title = route['title']
                route_link = route['link']
                transport = route['Transport']
                try:
                    bus_items = extract_all_bus_items(driver, wait, route_link, route_title, transport)
                    if bus_items:
                        all_bus_details.extend(bus_items)
                        print(f"\nBus Details for {route_title} done:")
                        print(pd.DataFrame(bus_items).head())
                except Exception as e:
                    print("Error:", e)
        except Exception as e:
            print("Error:", e)
except Exception as e:
    print("Error:", e)
finally:
    driver.quit()

#to convert dataframe
df = pd.DataFrame(all_bus_details)

#conert into csv file
path = "D:\\CHITRA\\REDBUS\\redbusdetails.csv"
df.to_csv(path, index=False)

Loading, please wait...

Bus Details for Pune to Goa done:
                                      Transport   route_name  \
0  Kadamba Transport Corporation Limited (KTCL)  Pune to Goa   
1  Kadamba Transport Corporation Limited (KTCL)  Pune to Goa   
2  Kadamba Transport Corporation Limited (KTCL)  Pune to Goa   
3  Kadamba Transport Corporation Limited (KTCL)  Pune to Goa   
4  Kadamba Transport Corporation Limited (KTCL)  Pune to Goa   

                                      route_link        Date  \
0  https://www.redbus.in/bus-tickets/pune-to-goa  2024-07-23   
1  https://www.redbus.in/bus-tickets/pune-to-goa  2024-07-23   
2  https://www.redbus.in/bus-tickets/pune-to-goa  2024-07-23   
3  https://www.redbus.in/bus-tickets/pune-to-goa  2024-07-23   
4  https://www.redbus.in/bus-tickets/pune-to-goa  2024-07-23   

               busname                        bustype departing_time duration  \
0        Atmaram Gobus           VE A/C Sleeper (2+1)          21:00  10h 30m   
1        

In [1]:
!pip install pymysql

In [2]:
import pymysql

In [ ]:
#python mysql connection

In [1]:
import pandas as pd
import pymysql
import re

# Function to clean data
def clean_data(value, data_type):
    if pd.isna(value):  # Handle NaN values
        return None
    if data_type == 'int':
        return int(''.join(filter(str.isdigit, str(value))))
    elif data_type == 'float':
        return float(''.join(filter(lambda x: x.isdigit() or x == '.', str(value))))
    else:
        return value

# Function to convert duration to 'HH:MM:SS' format
def convert_duration(duration):
    if pd.isna(duration):  # Handle NaN values
        return None
    match = re.match(r'(\d+)h\s*(\d+)m', duration)
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2))
        return f'{hours:02}:{minutes:02}:00'
    return None  # Return None if format does not match

# Read CSV file
csv_file_path = r"D:\CHITRA\REDBUS\redbusdetails.csv"
df = pd.read_csv(csv_file_path)

# Connect to MySQL
connection = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='2210',
    database='redbusproject'
)
cursor = connection.cursor()

# Create table
create_table_query = """
CREATE TABLE IF NOT EXISTS redbusdetails (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Transport VARCHAR(255),
    route_name VARCHAR(255),
    route_link VARCHAR(255),
    Date date,
    busname VARCHAR(255),
    bustype VARCHAR(255),
    departing_time TIME,
    duration TIME,
    bp_time TIME,
    star_rating FLOAT,
    price INT,
    seats_available INT
);
"""
cursor.execute(create_table_query)

# Ensure all NaNs are replaced with None
df = df.where(pd.notnull(df), None)

# Insert the extracted rows into the MySQL table
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO redbusdetails (Transport, route_name, route_link, Date, busname, bustype, departing_time, duration, bp_time, star_rating, price, seats_available)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    row_data = (
        row['Transport'],
        row['route_name'],
        row['route_link'],
        row['Date'],
        row['busname'],
        row['bustype'],
        row['departing_time'],
        convert_duration(row['duration']),
        row['bp_time'],
        clean_data(row['star_rating'], 'float'),
        clean_data(row['price'], 'int'),
        clean_data(row['seats_available'], 'int')
    )
    
    try:
        cursor.execute(insert_query, row_data)
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        print(f"Row data: {row_data}")

# Commit and close the connection
connection.commit()
cursor.close()
connection.close()

print("Data inserted successfully!")


Data inserted successfully!
